In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
orders_df = pd.read_csv("orders.csv")
orders_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users_df = pd.read_json("users.json")
users_df.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
db_conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r", encoding="utf-8") as sql_file:
    db_conn.executescript(sql_file.read())

restaurants_df = pd.read_sql("SELECT * FROM restaurants", db_conn)

restaurants_df.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
analytics_df = (
    orders_df
    .merge(users_df, on="user_id", how="left")
    .merge(restaurants_df, on="restaurant_id", how="left")
)

analytics_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
analytics_df.rename(
    columns={"total_amount": "order_amount"},
    inplace=True
)

In [7]:
analytics_df.drop(columns=["restaurant_name_x"], inplace=True)
analytics_df.rename(columns={"restaurant_name_y": "restaurant_name"}, inplace=True)

In [8]:
analytics_df["order_date"] = pd.to_datetime(
    analytics_df["order_date"],
    format="%d-%m-%Y"
)
analytics_df["order_month"] = analytics_df["order_date"].dt.month
analytics_df["order_year"] = analytics_df["order_date"].dt.year
analytics_df["order_day"] = analytics_df["order_date"].dt.day_name()

In [9]:
analytics_df["order_value_band"] = pd.cut(
    analytics_df["order_amount"],
    bins=[0, 200, 500, 1000, np.inf],
    labels=["Low Spend", "Mid Spend", "High Spend", "Premium Spend"]
)

In [10]:
user_frequency = analytics_df.groupby("user_id")["order_id"].count()

analytics_df["user_order_count"] = analytics_df["user_id"].map(user_frequency)

analytics_df["loyalty_group"] = pd.cut(
    analytics_df["user_order_count"],
    bins=[0, 2, 5, np.inf],
    labels=["New User", "Returning User", "Loyal User"]
)

In [11]:
analytics_df.groupby("membership")["order_amount"].sum()

membership
Gold       3975364.89
Regular    4036259.23
Name: order_amount, dtype: float64

In [12]:
analytics_df.groupby("city")["order_amount"].mean().sort_values(ascending=False)

city
Chennai      806.202118
Hyderabad    803.985779
Bangalore    802.234308
Pune         792.097914
Name: order_amount, dtype: float64

In [13]:
analytics_df.groupby("cuisine")["order_amount"].agg(
    total_revenue="sum",
    avg_order_value="mean"
).sort_values("total_revenue", ascending=False)

,total_revenue,avg_order_value
cuisine,,
Mexican,2085503.09,808.021344
Italian,2024203.80,799.448578
Indian,1971412.58,798.466011
Chinese,1930504.65,798.389020


In [14]:
analytics_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)
print("Final analytics dataset generated successfully")

Final analytics dataset generated successfully
